# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
import pickle

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/sajith/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sajith/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sajith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("select * from InsertTableName", engine)

In [29]:
df['genre'].value_counts()

news      13036
direct    10634
social     2358
Name: genre, dtype: int64

In [4]:
df.isna().any()

id                         True
message                   False
original                   True
genre                     False
related                    True
request                   False
offer                     False
aid_related               False
medical_help              False
medical_products          False
search_and_rescue         False
security                  False
military                  False
child_alone               False
water                     False
food                      False
shelter                   False
clothing                  False
money                     False
missing_people            False
refugees                  False
death                     False
other_aid                 False
infrastructure_related    False
transport                 False
buildings                 False
electricity               False
tools                     False
hospitals                 False
shops                     False
aid_centers               False
other_in

In [5]:
df = df.dropna(subset=['related'], axis=0)

In [6]:
df.isna().any()

id                         True
message                   False
original                   True
genre                     False
related                   False
request                   False
offer                     False
aid_related               False
medical_help              False
medical_products          False
search_and_rescue         False
security                  False
military                  False
child_alone               False
water                     False
food                      False
shelter                   False
clothing                  False
money                     False
missing_people            False
refugees                  False
death                     False
other_aid                 False
infrastructure_related    False
transport                 False
buildings                 False
electricity               False
tools                     False
hospitals                 False
shops                     False
aid_centers               False
other_in

In [7]:
# X = df.iloc[:,2]
# Y = df.iloc[:,4:]

In [8]:
X = df.loc[:, 'message']
Y = df.iloc[:, 4:]

In [36]:
X.values

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [27]:
for text in X.values:.str.split().apply(len).value_counts()
    print(text)

Weather update - a cold front from Cuba that could pass over Haiti
Is the Hurricane over or is it not over
Looking for someone but no name
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
says: west side of Haiti, rest of the country today and tonight
Information about the National Palace-
Storm at sacred heart of jesus
Please, we need tents and water. We are in Silo, Thank you!
I would like to receive the messages, thank you
I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )
There's nothing to eat and water, we starving and thirsty.
I am in Petionville. I need more information regarding 4636
I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks
Let's do it together, need food in Delma 75, in didine area
More information on the 4636 number in order for me to participate. ( To

What should people do when their house was cracked but hasn't fallen down in an area where the earthquake wasn't that strong? Answer please. 
Get this message through - there are people in Fort Jacques on Route Kalbas who don't have a place to live and who don't have any food. What can we do in this case? 
We are the people they forgot in Haiti, we counted many hundreds of people who are in need, we made an organisation so that our voice can be heard, we are .. 
I lost my mother ,my brother, my sister,I went to La gonave,I am not from there I ask what I am doing there 
..especially with PAM. For help with food, we are in the 5 communal sections of Petion Ville in Montagne Noire. This is OJDSC 
Good morning, I live in Port-au-Prince, I would like to know the address of the organization CHF. 
garbledwhat is available? there are 190 of us. thank you in advance. we hope for a fast response, things are getting critical here. 
or a carpet with a blanket to have shelter from dew and rain, if 

the big guys take the prelate to sell for 250 gourdes at delmas 32 and the street boukman don't find it. 
please, send me some minutes on my cell phone, because i want to do a urgent call. 
yes there's a old person who never find medicament 
if he/it pleases you my brothers help me has find an uses so that I can save the life of my brothers and sisters who are in difficulty 
I apologize for this response. Can you help me understand the SMS? Please send me information. Thank you. 
Digicel did have anybody died? 
good evening please can I know what will be tomorrow? 
What do people in the street have to do because It's rainning while the earthquake is going on? 
I don't understand the message they send please explain me what they want to say 
what information that it has some concerning the waters that one drinks. we are has the plain zones small ginen one doesn't have found nothing of the all.beceause that one doesn't have anybody. 
I'm ok in the name of jesus and you? 
i would like to 

RT gillo Eyewitness Tweets from Haiti http bit.ly 5QVS4c #haiti #earthquake
RBS Worldpay payment processor beweert down te zijn door earthquake . Tot zover de high availability. #diemoetikonthouden
for LIVE TWEETING from haiti follow fredodupoux carelpedre InternetHaiti photomorel #haiti #eq #haitiquake #earthquake
Anxious Haitians in U.S. Search for Information The devastating earthquake left Haitian Americans scrambling for .. http bit.ly 7tjHTT
RT DJWILLIE SANTO DOMINGO YOU TUBE VIDEO ON THE EARTHQUAKE http short.to 13f1u &lt &lt prayers going out!
pic.com xvzzt RT backblocks RT BADGIRLSTREETS smh Haiti Presidential Palace before and after earthquake!Pl cont http tl.gd 3g8mk
RT gotopuertorico UNICEF urgently appeals for aid for #Haiti following devastating earthquake http bit.ly 5ThUOF Port Au Prince RE ..
Praying for the people in #Haiti that have been hit by a disastrous earthquake.
RT ashong OMG Haiti gets hit by a record 7.0 earthquake?! Haven t these folks suffered enuff alread

It will also pilot a community-based rehabilitation model that can be replicated at the national level to serve more people in need of rehabilitative therapy and to better position and prepare China to respond to future disasters.
Notwithstanding the difficulties resulting from Russia's socio-economic collapse, the plight of the Yakutia people is even more stark in the context of the bitterly cold winter which is already upon them.
The National De-mining Commission has expressed concern about landmines, which have been dislodged by torrential floods.
The president commended the efforts of private citizens to provide assistance such as food and blankets, but said it is most important to provide cash donations to organizations that are able to focus resources and meet specific needs on the ground.
With an allocation of $100,000, the UN Development Programme (UNDP) is providing tarpaulins and other essential non- food items to improve shelter conditions for 865 displaced families.
This ha

In 2003 grain harvest in the Sahel reached 14 million metric tons (MT).
In the worst affected province, the northeastern province of Sylhet, rivers continued to overflow, leaving tens of thousands of families stranded on hundreds of tiny islands created by the deluge, the Disaster Management Office said.
I might visit a milk collection center to verify how much milk a loan applicant is selling, or I might visit a loan recipient to confirm that he has purchased his cows."
As winter approaches, with nighttime temperatures already reaching the freezing point, homeless children and their families could face snow and frost in the next few weeks.
Transportation to affected villages in Jayawijaya is a problem so far because the thick haze blanketing Kuyawage has forced missionary pilots to cancel several flight plans to the area.
Over one hundred shelters will be constructed this year before winter arrives from UNDP core resources.
Rains started pounding Zhouqu at around 10 p.m., which, accor

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1.0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
Y['request'] = Y['request'].astype('int')

### 2. Write a tokenization function to process your text data

In [12]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [13]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)



In [14]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
def build_pipeline(estimator):
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', estimator)
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 0)

In [17]:
multi_label_estimators = []
multi_label_estimator1 = MultiOutputClassifier(OneVsRestClassifier(SGDClassifier(loss='modified_huber', penalty='elasticnet',
                                          alpha=1e-4,  random_state=42,
                                          shuffle=True, n_jobs=5)))
multi_label_estimators.append(multi_label_estimator1)

multi_label_estimator2 = MultiOutputClassifier(OneVsRestClassifier(MultinomialNB()))
multi_label_estimators.append(multi_label_estimator2)

multi_label_estimator3 = MultiOutputClassifier(OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=5))
multi_label_estimators.append(multi_label_estimator3)

multi_label_estimator4 = MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier(n_jobs = 5)))
multi_label_estimators.append(multi_label_estimator4)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
for i , estimator in enumerate(multi_label_estimators):
    pipeline = build_pipeline(estimator)
    pipeline.fit(X_train, Y_train)
    prediction = pipeline.predict(X_test)
    print(classification_report(Y_test, prediction, target_names=Y.columns.values))

In [18]:
estimator = MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier(n_jobs = 5)))
pipeline = build_pipeline(estimator)

In [19]:
pipeline.fit(X_train, Y_train)

/home/sajith/Documents/Acedamic/Projects/Python3.6/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 0.0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [20]:
prediction = pipeline.predict(X_test)

In [23]:
print(classification_report(Y_test, prediction, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      3983
               request       0.91      0.44      0.60       931
                 offer       0.00      0.00      0.00        20
           aid_related       0.79      0.61      0.69      2226
          medical_help       0.71      0.03      0.06       447
      medical_products       0.64      0.06      0.10       284
     search_and_rescue       0.64      0.05      0.10       170
              security       1.00      0.01      0.02       104
              military       1.00      0.03      0.06       189
           child_alone       0.00      0.00      0.00         0
                 water       0.96      0.20      0.33       371
                  food       0.84      0.40      0.54       597
               shelter       0.83      0.31      0.45       477
              clothing       0.71      0.06      0.12        77
                 money       0.67      

In [ ]:
pickle.dump(pipeline, open('classifier.pkl', 'wb'))

In [ ]:
estimator.get_params()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 10, 100),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__estimator__n_estimators': [100, 200, 1000],
        'clf__estimator__estimator__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }
cv = GridSearchCV(pipeline, param_grid = parameters)

In [ ]:
cv.fit(X_train,Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.